## Pipeline de Consulta Semântica com RAG e LLMs Locais (Documentos SEI UTFPR)

Este pipeline foi projetado para **responder a perguntas sobre o conteúdo de documentos HTML do SEI UTFPR** de forma inteligente, utilizando a arquitetura de Recuperação Aumentada por Geração (RAG). Ele combina a **busca semântica em um banco vetorial ChromaDB** (contendo embeddings de documentos SEI) com a capacidade de **geração de respostas de Large Language Models (LLMs) locais**, otimizando a recuperação e a síntese de informações.

---

### Ferramentas Essenciais

* **sentence-transformers**: Para a criação de embeddings da consulta do usuário e o carregamento dos modelos de embedding.
* **ChromaDB**: Banco de dados vetorial para armazenamento e busca eficiente das informações.
* **transformers**: Para o carregamento e uso de Large Language Models (LLMs) locais na geração de respostas.
* **dotenv** e **huggingface_hub**: Para gerenciamento de autenticação no Hugging Face e variáveis de ambiente.
* **torch**: Biblioteca para computação tensorial e gerenciamento de hardware (GPU/CPU).

---

### Estratégia de Operação

1.  **Autenticação e Configuração Inicial**:
    * **Login no Hugging Face**: Autenticação via token para acesso a modelos restritos.
    * **Configuração do ChromaDB**: Inicialização do cliente persistente do ChromaDB, apontando para o diretório onde as coleções de embeddings estão armazenadas.

2.  **Seleção Dinâmica do Modelo de Embedding**:
    * O pipeline apresenta uma lista de **modelos de embedding disponíveis** (ex: `all-MiniLM-L6-v2`, `bert-base-portuguese-cased`).
    * O usuário escolhe qual modelo de embedding deseja utilizar para a consulta atual.
    * O **modelo de embedding e a coleção correspondente no ChromaDB são carregados sob demanda**, otimizando o uso de recursos de memória, pois apenas o necessário é mantido ativo.

3.  **Carregamento do LLM (Local)**:
    * Um **LLM local** (ex: `gemma-2b`, `llama-3.2-1b`) é carregado e permanece em memória.
    * A configuração de quantização (4-bit) com offload para CPU é utilizada para **otimizar o consumo de memória**, permitindo a execução em ambientes com recursos limitados de GPU.

4.  **Recuperação de Informações (RAG)**:
    * **Embeddings da Consulta**: A pergunta do usuário é convertida em um embedding usando o modelo de embedding selecionado.
    * **Extração de Filtros**: O pipeline tenta **identificar automaticamente termos nos metadados** (ex: `tipo_documento`, `órgão`, `data_publicação`, `título`) presentes na pergunta do usuário. Esses termos são usados como **filtros de metadados** para refinar a busca no ChromaDB.
    * **Busca no ChromaDB**: A busca é realizada no ChromaDB utilizando o embedding da consulta e os filtros de metadados extraídos. Isso garante que os **chunks mais relevantes e contextualmente alinhados** à pergunta sejam recuperados dos documentos SEI.

5.  **Geração de Resposta**:
    * **Criação do Prompt**: Os chunks recuperados servem como **contexto** para a construção de um prompt para o LLM. O prompt instrui o LLM a responder "APENAS com as informações fornecidas no contexto", atuando como um assistente especializado em documentos oficiais da UTFPR.
    * **Inferência do LLM**: O LLM local gera uma resposta coerente e concisa com base no prompt e no contexto fornecido, evitando informações externas ou "alucinações".

---

### Benefícios

* **Flexibilidade e Comparação**: Permite testar e comparar a performance de diferentes modelos de embedding em tempo real.
* **Precisão Contextual**: Utiliza filtros inteligentes baseados em metadados para garantir que as respostas sejam altamente relevantes ao contexto da pergunta sobre os documentos SEI.
* **Eficiência de Recurso**: Otimiza o uso de memória ao carregar modelos de embedding sob demanda e usar quantização para o LLM.
* **Transparência**: Exibe os chunks recuperados, seus metadados e o prompt enviado ao LLM, proporcionando visibilidade sobre o processo de recuperação e geração.
* **Controle de Informação**: As respostas são estritamente baseadas no contexto recuperado dos documentos SEI, garantindo factualidade.

In [ ]:
# Célula 1: Importando bibliotecas

# Bibliotecas padrão
import os
import re
import time
import json
import gc
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv

# Bibliotecas de dados e visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Bibliotecas de IA e embeddings
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import chromadb

# Bibliotecas de NLP e ML
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Bibliotecas para avaliação RAGAS
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
from ragas import evaluate
import ragas
from datasets import Dataset

# nltk.download('all') # Baixar recursos do NLTK (descomente se for a primeira execução)

In [ ]:
# Célula 2: Login no Hugging Face

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()
KEY = os.getenv('HUGGINGFACE_TOKEN')

# Fazer login no Hugging Face
from huggingface_hub import login
login(token=KEY)

In [ ]:
# Célula 3: Configuração do ChromaDB e mapeamento de modelos

# Configurar ChromaDB
CHROMA_DIR = "chroma_db"
chroma_client = chromadb.PersistentClient(path=CHROMA_DIR)

# Mapeamento dos nomes das coleções para os nomes corretos dos modelos Hugging Face
EMBEDDING_MODEL_MAP = {
    "all-MiniLM-L6-v2": "sentence-transformers/all-MiniLM-L6-v2",
    "all-mpnet-base-v2": "sentence-transformers/all-mpnet-base-v2",
    "paraphrase-multilingual-MiniLM-L12-v2": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "distiluse-base-multilingual-cased-v2": "sentence-transformers/distiluse-base-multilingual-cased-v2",
    "stsb-xlm-r-multilingual": "sentence-transformers/stsb-xlm-r-multilingual",
    "neuralmind-bert-base-portuguese-cased": "neuralmind/bert-base-portuguese-cased"
}

def load_embedding_model_and_collection(model_key: str):
    """
    Carrega o modelo de embedding e a collection correspondente.
    """
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_path = EMBEDDING_MODEL_MAP[model_key]
        collection_name = f"documentos-SEI__{model_key}"

        print(f"\nCarregando modelo: {model_path}")
        embedding_model = SentenceTransformer(model_path, device=device)

        print(f"Carregando collection: {collection_name}")
        collection = chroma_client.get_collection(collection_name)

        print(f"Collection carregada com {collection.count()} documentos.")
        return embedding_model, collection

    except Exception as e:
        print(f"Erro ao carregar modelo/collection: {str(e)}")
        return None, None

### Configurações Recomendadas para Carregamento de Modelos

Configurações recomendadas para carregar diferentes modelos de linguagem (LLMs) usando a biblioteca `transformers` do Hugging Face, visando otimizar compatibilidade, desempenho e estabilidade.

#### LLaMA, Gemma e outros modelos com quantização 4-bit

- **Configuração:**
  - `quantization_config = BitsAndBytesConfig(...)`: Configuração para quantização 4-bit, com parâmetros:
    - `load_in_4bit=True`: Ativa a quantização 4-bit.
    - `bnb_4bit_compute_dtype=torch.float16`: Define o tipo de dado para cálculos internos.
    - `llm_int8_enable_fp32_cpu_offload=True`: Permite offload para CPU em float32 para estabilidade.
  - `tokenizer = AutoTokenizer.from_pretrained(model_id)`: Carrega o tokenizer do modelo.

  - `model = AutoModelForCausalLM.from_pretrained(...)`: Carrega o modelo com:
    - `device_map="auto"`: Distribui o modelo automaticamente entre CPU/GPU.
    - `torch_dtype=torch.float16`: Usa half-precision para acelerar a inferência.
    - `quantization_config=quantization_config`: Aplica a quantização 4-bit.
    - `trust_remote_code=True`: Permite execução de código customizado do repositório.

- **Referências:**
  - [Documentação do `transformers` sobre quantização 4-bit](https://huggingface.co/docs/transformers/main/en/main_classes/quantization)

In [ ]:
# Célula 4: Carregar o LLM
AVAILABLE_MODELS = {
    "llama-3.2-1b": "meta-llama/Llama-3.2-1B",
    "gemma-2b": "google/gemma-2b-it", # Gemma 2B Original
    "phi-3-mini": "microsoft/Phi-3-mini-4k-instruct" # Phi-3-mini
}

def load_model(model_name: str):
    if model_name not in AVAILABLE_MODELS:
        raise ValueError(f"Modelo {model_name} não disponível. Escolha entre: {list(AVAILABLE_MODELS.keys())}")
    model_id = AVAILABLE_MODELS[model_name]
    print(f"\nCarregando modelo: {model_id}")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        llm_int8_enable_fp32_cpu_offload=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        trust_remote_code=True
    )
    return tokenizer, model

# Carregando o LLM (este permanece carregado pois é usado em todas as consultas)
selected_model = "llama-3.2-1b"  # Escolha o modelo desejado
tokenizer, llm_model = load_model(selected_model)

### Estratégia do Pipeline RAG: Detecção Automática de Filtros e Geração Especializada para Documentos Institucionais

#### 1. Extração Dinâmica de Filtros (`extract_metadata_filters`)
A função analisa a pergunta e os metadados disponíveis para criar filtros automáticos.

* **Amostragem Inteligente:** Obtém uma amostra de documentos para identificar os campos de metadados disponíveis.
* **Detecção Contextual:** Verifica se termos dos metadados como 'tipo_documento', 'órgão', 'data_publicação' e 'título' aparecem na pergunta do usuário.
* **Composição Lógica:** Cria filtros simples ou compostos (usando operadores `$eq` e `$and`) dependendo da quantidade de correspondências encontradas.

#### 2. Recuperação com Filtros Automáticos (`retrieve_relevant_chunks`)
Com os filtros identificados, a busca combina relevância semântica e filtragem por metadados.

1. **Busca Híbrida:** Aplica simultaneamente a busca por similaridade vetorial e os filtros de metadados detectados.
2. **Mecanismo de Fallback:** Se a busca com filtros falhar, recorre automaticamente à busca puramente semântica.
3. **Resultado:** Retorna os documentos mais relevantes que satisfazem tanto a semântica da pergunta quanto os critérios específicos mencionados.

#### 3. Geração Especializada para Documentos Oficiais (`create_prompt` e `generate_response`)
O sistema gera respostas adaptadas ao contexto institucional.

* **Persona Especializada:** Define o assistente como "especializado em documentos oficiais da UTFPR".
* **Instrução de Aprofundamento:** Inclui orientação para "entender melhor o que diz o contexto, para fornecer informações extras".
* **Parâmetros de Criatividade Controlada:** Utiliza `temperature=0.7` e `repetition_penalty=1.2` para equilibrar precisão e fluidez nas respostas sobre documentos oficiais.

In [ ]:
# Célula 5: Funções RAG para documentos SEI

def extract_metadata_filters(query: str, collection, embedding_model) -> dict:
    """
    Extrai possíveis filtros de metadados da query do usuário.
    Considera os campos típicos dos metadados dos chunks SEI.
    """
    dummy_embedding = embedding_model.encode("dummy texto").tolist()
    sample_results = collection.query(
        query_embeddings=[dummy_embedding],
        n_results=100
    )

    filters = []
    metadata_fields = ['tipo_documento', 'órgão', 'data_publicação', 'título']

    for meta in sample_results['metadatas'][0]:
        for field in metadata_fields:
            if field in meta:
                value = meta[field]
                if value and value.lower() in query.lower():
                    filters.append({"$eq": {field: value}})

    if len(filters) == 1:
        return filters[0]  # Ex: {"$eq": {"tipo_documento": "Portaria"}}
    elif len(filters) > 1:
        return {"$and": filters}
    else:
        return None

def retrieve_relevant_chunks(query: str, embedding_model, collection, n_results: int = 5):
    """
    Recupera os chunks mais relevantes do ChromaDB usando similarity search e filtros automáticos.
    """
    query_embedding = embedding_model.encode(query).tolist()
    where_filter = extract_metadata_filters(query, collection, embedding_model)
    if where_filter:
        print("\nFiltros detectados:", where_filter)

    try:
        if where_filter:
            results = collection.query(
                query_embeddings=[query_embedding],
                where=where_filter,
                n_results=n_results,
                include=["documents", "metadatas"]
            )
        else:
            results = collection.query(
                query_embeddings=[query_embedding],
                n_results=n_results,
                include=["documents", "metadatas"]
            )
    except Exception as e:
        print(f"Erro na busca: {e}")
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            include=["documents", "metadatas"]
        )

    return results

def create_prompt(query: str, context: str) -> str:
    """
    Cria um prompt para o LLM usando a query e o contexto.
    """
    prompt = (
        f"Você é um assistente especializado em documentos oficiais da UTFPR. "
        f"Responda à pergunta usando APENAS as informações fornecidas no contexto abaixo.\n"
        f"Se a informação não estiver disponível no contexto, diga que não pode responder.\n"
        f"Se houver múltiplas informações relacionadas, forneça uma resposta completa e organizada.\n"
        f"Tente entender melhor o que diz o contexto, para fornecer informações extras.\n"
        f"---\n"
        f"Contexto:\n{context}\n"
        f"---\n"
        f"Pergunta: {query}\n"
        f"Resposta:"
    )
    return prompt

def generate_response(prompt: str, tokenizer, model, max_length: int = 4096) -> str:
    """
    Gera uma resposta usando o LLM.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        min_length=100,
        repetition_penalty=1.2,
        temperature=0.7,
        do_sample=True
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.replace(prompt, "").strip()

In [ ]:
# Célula 6: Interface de consulta para documentos SEI (com exibição explícita dos metadados)

def select_embedding_model():
    """
    Permite ao usuário selecionar o modelo de embedding desejado.
    """
    print("\nModelos de embedding disponíveis:")
    for idx, (key, path) in enumerate(EMBEDDING_MODEL_MAP.items(), 1):
        print(f"{idx}. {key}")

    while True:
        try:
            choice = int(input("\nEscolha o número do modelo: "))
            if 1 <= choice <= len(EMBEDDING_MODEL_MAP):
                return list(EMBEDDING_MODEL_MAP.keys())[choice-1]
            print("Escolha inválida. Tente novamente.")
        except ValueError:
            print("Por favor, digite um número válido.")

def rag_query_with_model_selection():
    """
    Interface principal do RAG com seleção de modelo para documentos SEI.
    """
    # 1. Selecionar modelo
    model_key = select_embedding_model()

    # 2. Carregar modelo e collection
    embedding_model, collection = load_embedding_model_and_collection(model_key)
    if not embedding_model or not collection:
        print("Não foi possível carregar o modelo/collection.")
        return

    try:
        # 3. Fazer a consulta
        query = input("\nDigite sua pergunta: ")

        # 4. Executar o RAG
        print("\nProcessando pergunta:", query)
        start_time = time.time()

        results = retrieve_relevant_chunks(query, embedding_model, collection)
        if not results['documents'][0]:
            print("Nenhum resultado encontrado.")
            return

        context = "\n\n".join(results['documents'][0])
        first_metadata = results['metadatas'][0][0] if results['metadatas'][0] else {}

        prompt = create_prompt(query, context)
        response = generate_response(prompt, tokenizer, llm_model)

        # 5. Mostrar resultados
        print("\n" + "="*50)
        print("Chunks recuperados:")
        for doc, meta in zip(results['documents'][0], results['metadatas'][0]):
            print("\nConteúdo:", doc.strip())
            print("Metadados:")
            for k, v in meta.items():
                print(f"  {k}: {v}")
            print("-"*40)

        # Mostrar metadados do primeiro chunk explicitamente
        print("\n=== Metadados do documento mais relevante ===")
        for k, v in first_metadata.items():
            print(f"{k}: {v}")

        print("\n=== Resposta do LLM ===")
        print(response)

        total_time = time.time() - start_time
        print("\n" + "="*50)
        print(f"Tempo total de processamento: {total_time:.2f} segundos")

    finally:
        # 6. Limpar memória
        print("\nLiberando memória...")
        del embedding_model
        gc.collect()
        print("Concluído!")

# Executar uma única consulta
rag_query_with_model_selection()

In [ ]:
# Célula 7: Avaliação do pipeline RAG para HTMLs

from tqdm.notebook import tqdm

def evaluate_rag_pipeline_html():
    """
    Avalia o pipeline RAG para HTMLs usando um conjunto de perguntas predefinidas.
    Salva os resultados em um arquivo JSON para posterior avaliação com RAGAS.
    """
    # 1. Carregar o JSON de perguntas (caminho atualizado para HTMLs)
    questions_path = "../RAGAS/03_Questions_HTML/Questions_HTML.json"
    print(f"Carregando perguntas de {questions_path}...")

    try:
        with open(questions_path, 'r', encoding='utf-8') as f:
            questions_data = json.load(f)
        print(f"Carregadas {len(questions_data)} perguntas.")
    except Exception as e:
        print(f"Erro ao carregar o arquivo de perguntas: {e}")
        return

    # 2. Selecionar o modelo de embedding
    print("\nModelos de embedding disponíveis:")
    for idx, (key, path) in enumerate(EMBEDDING_MODEL_MAP.items(), 1):
        print(f"{idx}. {key}")

    while True:
        try:
            choice = int(input("\nEscolha o número do modelo: "))
            if 1 <= choice <= len(EMBEDDING_MODEL_MAP):
                model_key = list(EMBEDDING_MODEL_MAP.keys())[choice-1]
                break
            print("Escolha inválida. Tente novamente.")
        except ValueError:
            print("Por favor, digite um número válido.")

    # 3. Carregar modelo e collection
    print(f"\nCarregando modelo de embedding: {model_key}")
    start_load = time.time()
    embedding_model, collection = load_embedding_model_and_collection(model_key)
    load_time = time.time() - start_load

    if not embedding_model or not collection:
        print("Não foi possível carregar o modelo/collection.")
        return

    print(f"Modelo e collection carregados em {load_time:.2f} segundos.")

    # 4. Processar cada pergunta e armazenar resultados
    results = []

    try:
        print("\nProcessando perguntas...")
        for i, question_item in enumerate(tqdm(questions_data)):
            question = question_item["question"]
            ground_truth = question_item["ground_truth"]

            print(f"\n\nPergunta {i+1}/{len(questions_data)}: {question}")

            # Recuperar chunks relevantes usando a função específica para HTMLs
            start_time = time.time()
            chunks_results = retrieve_relevant_chunks(question, embedding_model, collection)

            # Verificar se encontrou resultados
            if not chunks_results['documents'][0]:
                print("Nenhum resultado encontrado.")
                contexts = []
                answer = "Não foi possível encontrar informações relevantes para responder à pergunta."
            else:
                # Preparar contexto e gerar resposta
                contexts = chunks_results['documents'][0]
                context_text = "\n\n".join(contexts)
                prompt = create_prompt(question, context_text)
                answer = generate_response(prompt, tokenizer, llm_model)

            process_time = time.time() - start_time

            # Exibir resultados
            print("\nContextos recuperados:")
            for j, (doc, meta) in enumerate(zip(chunks_results['documents'][0], chunks_results['metadatas'][0])):
                print(f"\nContexto {j+1}:")
                print(f"Conteúdo: {doc.strip()}")
                print("Metadados:", meta)

            print("\nResposta gerada:")
            print(answer)

            print("\nGround Truth:")
            for gt in ground_truth:
                print(f"- {gt}")

            print(f"\nTempo de processamento: {process_time:.2f} segundos")

            # Armazenar resultados
            result_item = {
                "question": question,
                "answer": answer,
                "contexts": contexts,
                "ground_truth": ground_truth,
                "metadata": {
                    "embedding_model": model_key,
                    "process_time": process_time
                }
            }
            results.append(result_item)

    except Exception as e:
        print(f"Erro durante o processamento: {e}")

    finally:
        # 5. Salvar resultados em um arquivo JSON (caminho atualizado para HTMLs)
        # Criar diretório para o modelo se não existir
        output_dir = f"../RAGAS/03_Questions_HTML/{selected_model}/{model_key}"
        os.makedirs(output_dir, exist_ok=True)

        output_filename = f"{output_dir}/Answers_{model_key}_HTML.json"

        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\nResultados salvos em {output_filename}")
        except Exception as e:
            print(f"Erro ao salvar resultados: {e}")

        # 6. Limpar memória
        print("\nLiberando memória...")
        del embedding_model
        gc.collect()
        print("Avaliação concluída!")

# Executar a avaliação
evaluate_rag_pipeline_html()

In [ ]:
# Célula 8: Avaliação com RAGAS 0.2.15 usando OpenAI GPT-4o mini para HTMLs

warnings.filterwarnings("ignore")

# Carregar variáveis de ambiente do arquivo .env
from dotenv import load_dotenv
load_dotenv()

# Obter a chave da API OpenAI do arquivo .env
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("ERRO: Chave da API OpenAI não encontrada no arquivo .env")
    print("Por favor, crie um arquivo .env com o seguinte conteúdo:")
    print("OPENAI_API_KEY=sua-chave-api-aqui")
    raise ValueError("Chave da API OpenAI não encontrada")
else:
    print("Chave da API OpenAI carregada com sucesso do arquivo .env")
    # Definir explicitamente a variável de ambiente
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Configurar o modelo GPT-4o mini para o RAGAS
try:
    # Tentar usar LangchainLLM
    from ragas.llms import LangchainLLM
    from langchain_openai import ChatOpenAI

    # Criar instância do modelo OpenAI com a chave do .env
    chat_model = ChatOpenAI(
        model_name="gpt-4o-mini",  # Usando GPT-4o mini
        temperature=0,
        openai_api_key=openai_api_key
    )

    # Configurar RAGAS para usar este modelo
    ragas_llm = LangchainLLM(chat_model)
    ragas.llms.set_global_llm(ragas_llm)
    print("RAGAS configurado para usar gpt-4o-mini via LangchainLLM")
except Exception as e:
    print(f"Erro ao configurar LangchainLLM: {e}")
    print("Continuando com o modelo padrão do RAGAS")

# Definir o caminho base para os arquivos RAGAS
RAGAS_BASE_PATH = "../RAGAS"  # Um nível acima de TCC2/NLP/

def load_and_prepare_data(json_path):
    """
    Carrega os dados do JSON e os prepara para avaliação.
    """
    print(f"Carregando dados de {json_path}...")

    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Preparar dados no formato correto
        dataset_dict = {
            "user_input": [],
            "response": [],
            "retrieved_contexts": [],
            "reference": []
        }

        for item in data:
            dataset_dict["user_input"].append(item["question"])
            dataset_dict["response"].append(item["answer"])
            dataset_dict["retrieved_contexts"].append(item["contexts"])
            dataset_dict["reference"].append(" ".join(item["ground_truth"]))

        # Converter para Dataset do HuggingFace
        dataset = Dataset.from_dict(dataset_dict)
        print(f"Dados preparados com {len(dataset)} exemplos.")
        return dataset

    except Exception as e:
        print(f"Erro ao carregar ou preparar os dados: {e}")
        return None

def evaluate_with_ragas(dataset):
    """
    Avalia o dataset usando métricas do RAGAS.
    """
    print("Iniciando avaliação com RAGAS...")

    # Definir métricas (adicionando context_precision como quarta métrica)
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision
    ]

    try:
        print("Usando função evaluate() do RAGAS...")
        result = evaluate(
            dataset=dataset,
            metrics=metrics
        )

        print("Tipo do resultado:", type(result))
        print("Estrutura do resultado:", dir(result))

        # Extrair resultados com base no tipo de objeto retornado
        results = {}

        # Verificar se o resultado tem um atributo 'scores'
        if hasattr(result, 'scores'):
            print("Resultado tem atributo 'scores'")

            # Verificar se scores é uma lista
            if isinstance(result.scores, list):
                print("scores é uma lista com", len(result.scores), "elementos")

                # Imprimir o primeiro elemento para diagnóstico
                if result.scores:
                    print("Primeiro elemento de scores:", result.scores[0])

                    # Tentar extrair métricas do primeiro elemento
                    if hasattr(result.scores[0], 'name') and hasattr(result.scores[0], 'score'):
                        # Parece ser uma lista de objetos Score
                        for score_obj in result.scores:
                            metric_name = score_obj.name
                            score_value = score_obj.score
                            # Corrigir o nome da métrica para começar com 1 em vez de 0
                            if metric_name.endswith('_0'):
                                metric_name = metric_name[:-1] + '1'
                            results[metric_name] = score_value
                            print(f"{metric_name}: {score_value:.4f}")
                    else:
                        # Tentar extrair como dicionário
                        for i, score in enumerate(result.scores):
                            if isinstance(score, dict):
                                for metric_name, value in score.items():
                                    # Corrigir o índice para começar com 1 em vez de 0
                                    results[f"{metric_name}_{i+1}"] = value
                                    print(f"{metric_name}_{i+1}: {value:.4f}")
            else:
                # Se não for uma lista, tentar como dicionário
                try:
                    for metric_name, values in result.scores.items():
                        # Corrigir o nome da métrica para começar com 1 em vez de 0
                        if metric_name.endswith('_0'):
                            metric_name = metric_name[:-1] + '1'
                        results[metric_name] = values
                        avg_score = np.mean(values) if isinstance(values, list) else values
                        print(f"{metric_name}: {avg_score:.4f}")
                except Exception as e:
                    print(f"Erro ao processar scores como dicionário: {e}")

        # Verificar se o resultado tem um método 'to_pandas'
        elif hasattr(result, 'to_pandas'):
            print("Resultado tem método 'to_pandas'")
            df = result.to_pandas()
            print("Colunas do DataFrame:", df.columns.tolist())

            for col in df.columns:
                if col not in dataset.column_names:
                    # Corrigir o nome da coluna para começar com 1 em vez de 0
                    new_col = col
                    if col.endswith('_0'):
                        new_col = col[:-1] + '1'
                    results[new_col] = df[col].tolist()
                    avg_score = np.mean(df[col])
                    print(f"{new_col}: {avg_score:.4f}")

        # Se nenhuma das abordagens acima funcionar, tentar extrair informações básicas
        else:
            print("Formato de resultado desconhecido, tentando extrair informações básicas")
            # Tentar converter para string e extrair informações
            result_str = str(result)
            print("Resultado como string:", result_str[:500] + "..." if len(result_str) > 500 else result_str)

        # Verificar se conseguimos extrair algum resultado
        if not results:
            print("Não foi possível extrair resultados automaticamente.")
            print("Tentando acessar _scores_dict...")

            if hasattr(result, '_scores_dict'):
                try:
                    scores_dict = result._scores_dict
                    print("_scores_dict:", scores_dict)

                    if isinstance(scores_dict, dict):
                        for metric_name, values in scores_dict.items():
                            # Corrigir o nome da métrica para começar com 1 em vez de 0
                            if metric_name.endswith('_0'):
                                metric_name = metric_name[:-1] + '1'
                            results[metric_name] = values
                            avg_score = np.mean(values) if isinstance(values, list) else values
                            print(f"{metric_name}: {avg_score:.4f}")
                except Exception as e:
                    print(f"Erro ao processar _scores_dict: {e}")

        return results
    except Exception as e:
        print(f"Erro ao avaliar com RAGAS: {e}")
        import traceback
        traceback.print_exc()
        return {}

def visualize_results(results, full_model_name):
    """
    Visualiza os resultados da avaliação e salva na subpasta do embedding.
    """
    if not results:
        print("Sem resultados para visualizar.")
        return

    # Extrair o nome do modelo LLM e do embedding
    if "/" in full_model_name:
        llm_name, embedding_name = full_model_name.split("/", 1)
    else:
        # Compatibilidade com o formato antigo
        llm_name = "unknown_llm"
        embedding_name = full_model_name

    # Título para o gráfico
    display_name = f"{llm_name} + {embedding_name}"

    # Preparar dados para visualização
    metrics = []
    scores = []

    for metric, values in results.items():
        if values is not None:
            metrics.append(metric)
            # Calcular média se for uma lista, caso contrário usar o valor diretamente
            if isinstance(values, list):
                scores.append(np.mean(values))
            else:
                scores.append(values)

    if not metrics:
        print("Sem métricas para visualizar.")
        return

    # Criar DataFrame
    df = pd.DataFrame({
        "Métrica": metrics,
        "Pontuação": scores
    })

    # Plotar gráfico
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Métrica", y="Pontuação", data=df)
    plt.title(f"Avaliação RAGAS - {display_name}")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Caminho para a subpasta do embedding dentro da pasta do modelo LLM
    # Alterado para usar a pasta 03_Questions_HTML
    embedding_dir = os.path.join(RAGAS_BASE_PATH, "03_Questions_HTML", llm_name, embedding_name)
    os.makedirs(embedding_dir, exist_ok=True)

    # Salvar figura com o nome correto na subpasta do embedding
    output_file = os.path.join(embedding_dir, f"Answers_{embedding_name}_ragas_results.png")
    plt.savefig(output_file)
    plt.show()
    print(f"Gráfico salvo em {output_file}")

    # Salvar resultados em CSV com o nome correto na subpasta do embedding
    csv_file = os.path.join(embedding_dir, f"Answers_{embedding_name}_ragas_results.csv")
    df.to_csv(csv_file, index=False)
    print(f"Resultados salvos em {csv_file}")

def run_ragas_evaluation_html():
    """
    Função principal para executar a avaliação RAGAS para HTMLs.
    """
    # Definir o diretório base de respostas para HTMLs
    questions_dir = os.path.join(RAGAS_BASE_PATH, "03_Questions_HTML")

    # Verificar se o diretório existe
    if not os.path.exists(questions_dir):
        print(f"ERRO: Diretório {questions_dir} não encontrado.")
        print("Verifique se o caminho está correto.")
        return

    # Verificar se a variável selected_model está definida no escopo global
    global selected_model

    # Verificar se a variável existe e tem um valor
    if 'selected_model' not in globals() or not selected_model:
        print("AVISO: Variável 'selected_model' não encontrada ou vazia.")
        print("\nModelos LLM disponíveis:")
        for idx, model_name in enumerate(AVAILABLE_MODELS.keys(), 1):
            print(f"{idx}. {model_name}")

        while True:
            try:
                model_choice = int(input("\nEscolha o número do modelo LLM para avaliar: "))
                if 1 <= model_choice <= len(AVAILABLE_MODELS):
                    selected_model = list(AVAILABLE_MODELS.keys())[model_choice-1]
                    break
                print("Escolha inválida. Tente novamente.")
            except ValueError:
                print("Por favor, digite um número válido.")

    print(f"Usando modelo LLM: {selected_model}")

    # Verificar se a pasta do modelo LLM existe
    llm_dir = os.path.join(questions_dir, selected_model)
    if not os.path.exists(llm_dir):
        print(f"ERRO: Diretório do modelo LLM '{selected_model}' não encontrado em {questions_dir}")
        print(f"Criando diretório {llm_dir}")
        os.makedirs(llm_dir, exist_ok=True)
        print("Nenhum embedding encontrado para este modelo.")
        return

    # Listar as pastas de embedding dentro do diretório do modelo LLM
    embedding_folders = [d for d in os.listdir(llm_dir)
                         if os.path.isdir(os.path.join(llm_dir, d))]

    if not embedding_folders:
        print(f"Nenhuma pasta de embedding encontrada para o modelo {selected_model}")
        return

    print("\nModelos de embedding disponíveis:")
    for idx, embedding_name in enumerate(embedding_folders, 1):
        print(f"{idx}. {embedding_name}")

    # Selecionar modelo de embedding para avaliação
    while True:
        try:
            choice = int(input("\nEscolha o número do modelo de embedding para avaliar (0 para avaliar todos): "))
            if 0 <= choice <= len(embedding_folders):
                break
            print("Escolha inválida. Tente novamente.")
        except ValueError:
            print("Por favor, digite um número válido.")

    # Avaliar modelos de embedding selecionados
    if choice == 0:
        embeddings_to_evaluate = embedding_folders
    else:
        embeddings_to_evaluate = [embedding_folders[choice-1]]

    # Executar avaliação para cada modelo de embedding
    for embedding_name in embeddings_to_evaluate:
        # Caminho para a pasta do embedding
        embedding_dir = os.path.join(llm_dir, embedding_name)

        # Procurar o arquivo JSON de respostas na pasta do embedding
        # Alterado para procurar arquivos com _HTML.json
        json_files = [f for f in os.listdir(embedding_dir)
                     if f.endswith("_HTML.json")]

        if not json_files:
            print(f"Nenhum arquivo JSON encontrado para o embedding {embedding_name}")
            continue

        # Usar o primeiro arquivo JSON encontrado
        file = json_files[0]
        file_path = os.path.join(embedding_dir, file)

        print(f"\n{'='*50}")
        print(f"Avaliando {file} para o modelo LLM {selected_model} com embedding {embedding_name}...")

        # Carregar e preparar dados
        dataset = load_and_prepare_data(file_path)

        if dataset is None:
            continue

        # Avaliar com RAGAS
        results = evaluate_with_ragas(dataset)

        # Visualizar resultados - passando o caminho completo incluindo o modelo LLM
        full_model_name = f"{selected_model}/{embedding_name}"
        visualize_results(results, full_model_name)

        print(f"Avaliação de {embedding_name} com modelo LLM {selected_model} concluída!")

# Executar avaliação RAGAS para HTMLs
run_ragas_evaluation_html()

In [ ]:
# Célula: Exibir os 10 chunks mais relevantes

# Selecione o modelo de embedding desejado (ajuste se necessário)
model_key = "all-MiniLM-L6-v2"  # ou outro disponível no seu EMBEDDING_MODEL_MAP

# Carregue o modelo de embedding e a coleção correspondente
embedding_model, collection = load_embedding_model_and_collection(model_key)
if not embedding_model or not collection:
    print("Não foi possível carregar o modelo/collection.")
else:
    consulta = "Regulamentar as atividades complementares (ACs) dos cursos de graduação"
    print(f"Buscando chunks relevantes para: '{consulta}'")
    resultados = retrieve_relevant_chunks(consulta, embedding_model, collection, n_results=10)

    docs = resultados.get("documents", [[]])[0]
    metas = resultados.get("metadatas", [[]])[0]

    print(f"\nTotal de chunks retornados: {len(docs)}\n")
    for i, (doc, meta) in enumerate(zip(docs, metas), 1):
        print(f"Chunk {i}:")
        print("Conteúdo:", doc.strip())
        print("Metadados:", meta)
        print("-" * 60)

# Estes chunks foram utilizados para identificar e gerar perguntas e respostas sobre determinados documentos,
# permitindo avaliar a qualidade das respostas com o framework RAGAS.